 - Step 0: Label all client nodes as “not served”. Go to Step 1.
 - Step 1: If all client nodes are already labelled “served”, then STOP. Otherwise, start at one of the centres (or ‘depots’) with a new staff member, initialize the work duration of this staff member as zero, and go to Step 2.
 - Step 2: From the current node, pick the nearest neighbour client node j that is “not served”. If the current work duration and the walking distance to j add up to more than 7 hours, then this staff member has completed their shift, go to Step 1. Otherwise, go to Step 3.
 - Step 3: Add the walking distance to j as well as the task duration at j to the current work duration. Update the label of node j from “not served” to “served”. If the updated work duration is more than 7 hours, then this staff member has completed their shift, go to Step 1. Otherwise, go to Step 2.

Depots are 71, 142, 280, 3451, 6846, and 7649

In [1]:
!pip install ortools

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import itertools
import copy
import networkx as nx
import scipy as sci

In [2]:
CareDist_Matrix = np.loadtxt(open("CareDistances-FULL.csv", "rb"), dtype=int, delimiter=",", skiprows=1)
CareDist_Matrix

array([[   0,    4,   38, ..., 8360, 8507, 8545],
       [   4,    0, 1280, ...,  777, 1025, 1017],
       [  38, 1280,    0, ..., 1408, 1515, 1433],
       ...,
       [8360,  777, 1408, ...,    0, 1530, 1449],
       [8507, 1025, 1515, ..., 1530,    0,   82],
       [8545, 1017, 1433, ..., 1449,   82,    0]])

Replace all 0 distance values with a large number to prevent routing there due to the 0 distance

In [3]:
CareDist_Matrix[CareDist_Matrix == 0] = 10000000

In [4]:
CareDist_Matrix[0,0]=0

In [5]:
CareDist_Matrix[0,220]

7649

The centres correspond to the following nodes:
 - 71 is Node 3
 - 142 is Node 7
 - 280 is Node 14
 - 3451 is Node 104
 - 6846 is Node 185
 - 7649 is Node 220

In [14]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = CareDist_Matrix
    data['num_vehicles'] = 6
    data['starts'] = [3, 7, 14, 104, 185, 220]
    data['ends'] = [0, 0, 0, 0, 0, 0]
    return data

In [15]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

In [16]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['starts'],
                                           data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        25200,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [ ]:
if __name__ == '__main__':
    main()